In [2]:
from konlpy.tag import Okt
from gensim.models.fasttext import FastText, load_facebook_vectors, load_facebook_model
import os
import pandas as pd
import re
from gensim.test.utils import datapath

def preprocess_korean_text(text, stop_words=[]):
    okt = Okt()
    
    # 형태소 분석을 통한 토큰화
    words = okt.morphs(text, norm=True) # 정규화 처리
    
    # 불용어 제거
    words = [word for word in words if word not in stop_words]
    
    return words

def process_csv_file(csv_path, stop_words=[]):
    new_data = []
    
    df = pd.read_csv(csv_path, encoding='utf-8')
    if 'reviews' in df.columns:
        for text in df['reviews']:
            # 추가적인 전처리 작업 (예: 특수 문자 제거, 숫자 제거 등)
            text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text)
            
            # 전처리된 문장을 추가 데이터에 포함
            tokens = preprocess_korean_text(text, stop_words)
            new_data.append(tokens)
    
    return new_data


# 기존 pre-trained 모델 경로
pretrained_model_path = 'wiki.ko.bin'

# 새로운 데이터 파일 경로
csv_file_path = '../ReviewData/reviews_only.csv'

# 불용어 리스트 (필요에 따라 추가)
stop_words = ['의','가', '에','들','는','잘','걍','과','도','를','으로','한','하다','!','?','<','>','(',')','[',']','|','#','.', '이','은','는','을','에','에서','로']

# 모델 로드
# model = load_facebook_vectors(pretrained_model_path)
model = load_facebook_model(pretrained_model_path)

# CSV 파일 처리 및 전처리
new_data = process_csv_file(csv_file_path, stop_words)

# 모델 업데이트
model.build_vocab(corpus_iterable=new_data, update=True)
model.train(corpus_iterable=new_data, total_examples=len(new_data), epochs=model.epochs)

# 업데이트된 모델 저장
updated_model_path = 'updated_model.bin'
model.save(updated_model_path)

# 저장된 모델 경로 출력
print(f"모델이 성공적으로 업데이트되었습니다. 업데이트된 모델 경로: {os.path.abspath(updated_model_path)}")


KeyboardInterrupt



In [5]:
from gensim.models.fasttext import FastText

# 저장한 모델 불러오기
model = FastText.load('./updated_model.bin')

def find_similar_words(word, topn=10):
    try:
        # 입력한 단어와 유사한 단어들을 찾기
        similar_words = model.wv.most_similar(word, topn=topn)

        # 결과 출력
        print(f"입력한 단어 '{word}'와(과) 유사한 단어들:")
        for similar_word, similarity in similar_words:
            print(f"{similar_word}: {similarity}")

    except KeyError:
        print(f"입력한 단어 '{word}'는 모델의 어휘에 없습니다.")

# 사용자로부터 단어 입력 받기
user_input = input("단어를 입력하세요: ")

# 유사한 단어 찾기 및 출력
find_similar_words(user_input)

입력한 단어 '치킨'와(과) 유사한 단어들:
양념치킨: 0.8387115001678467
치킨과: 0.8261151909828186
치킨윙: 0.8219879865646362
치킨에: 0.819800078868866
치킨이: 0.809363842010498
치킨은: 0.7859618663787842
치킨을: 0.785783588886261
치킨의: 0.777712881565094
먼치킨: 0.7542405724525452
치킨집: 0.7345045804977417
